# ch12. 텐서플로를 사용한 사용자 정의 모델과 훈련

* 텐서플로우
- 고수준 API tf.keras -> 딥러닝 작업 중 95%에 필요
- 저수준 API (파이썬) -> 커스텀 손실 함수, 지표, 층, 모델, 초기화, 규제, 가중치 규제 등 세부 제어시 필요


## 12.1 텐서플로 훑어보기
## 12.2 넘파이처럼 텐서플로 사용하기
- 12.2.1 텐서와 연산
- 12.2.2 텐서와 넘파이
- 12.2.3 타입 변환 
- 12.2.4 변수
- 12.2.5 다른 데이터 구조

## 12.3 사용자 정의 모델과 훈련 알고리즘
- 12.3.1 사용자 정의 손실 함수
- 12.3.2 사용자 정의 요소를 가진 모델을 저장하고 로드하기
- 12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하지
- 12.3.4 사용자 정의 지표
- 12.3.5 사용자 정의층
- 12.3.6 사용자 정의 모델
- 12.3.7 모델 구성 요소에 기반한 손실과 지표
- 12.3.8 자동 미분을 사용하여 그래디언트 계산하기
- 12.3.9 사용자 정의 훈련 반복

## 12.4 텐서플로 함수와 그래프
- 12.4.1 오토그래프와 트레이싱
- 12.4.2 텐서플로 함수 사용 방법

**Chapter 12 – Custom Models and Training with TensorFlow**

_This notebook contains all the sample code in chapter 12._

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/12_custom_models_and_training_with_tensorflow.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 12.2 넘파이처럼 텐서플로 사용하기

* 텐서플로 API
- 텐서(tensor)를 순환(flow)시킴
- 텐서는 한 연산에서 다른 연산으로 흐름
- 넘파이 ndarray와 매우 비슷 -> 다차원 배열
- 스칼라 값도 가질 수 있음 

# 12.2.1 텐서와 연산

## Tensors and operations

### Tensors

In [5]:
# 텐서 만들기: tf.constant()
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

<tf.Tensor: id=0, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [7]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t

<tf.Tensor: id=1, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
# 텐서 크기(shpae) 확인
t.shape

TensorShape([2, 3])

In [6]:
# 텐서 데이터 타입 확인
t.dtype

tf.float32

### Indexing

- 인덱스 참조는 넘파이와 비슷

In [7]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

### Ops

- 모든 종류의 텐서 연산 가능

In [9]:
# tf.add(t, 10)과 동일
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [8]:
# 행렬 곱셉 연산
# tf.matmul()과 동일
t @ tf.transpose(t)

<tf.Tensor: id=4, shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

# 12.2.2 텐서와 넘파이

* 텐서 -> 넘파이 배열 만들기 가능
* 넘파이 배열 -> 텐서 만들기 가능
    - 넘파이: 64비트 정밀도
    - 텐서플로: 32비트 정밀도
        - 신경망은 32비트 정밀도로 충분, 더 빠르고, 더 적게 메모리 사용
        - 넘파이 배열로 텐서를 만들때 dtype=tf.float32로 지정해야
    
* 넘파이에 텐서 플로 연산 가능
* 텐서에 넘파이 연산 가능 

### From/To NumPy

In [16]:
# 넘파이 배열로 텐서 만들기
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: id=5, shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [17]:
# 텐서로 넘파이 배열 만들기
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [18]:
# 텐서로 넘파이 배열 만들기
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [19]:
# 넘파이 배열에 텐서플로 연산 적용
tf.square(a)

<tf.Tensor: id=7, shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [20]:
# 텐서에 넘파이 연산 적용
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

# 12.2.3 타입 변환

* 타입 변환은 성능을 크게 감소 시킬 수 있음 
* 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않음
* 호환되지 않은 타입의 텐서를 연산 수행하면 예외 발생
    - 실수 텐서 + 정수 텐서 -> 불가
    - 32비트 실수 + 64 비트 실수 -> 불가


### Conflicting Types

In [24]:
# 실수 텐서와 정수 텐서 더하기 -> 불가
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: add/


In [25]:
# 실수 텐서(기본 32비트)와 실수 텐서(64비트 변환) 더하기 -> 불가
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: add/


In [26]:
# tf.cast(): 타입 변환
t2 = tf.constant(40., dtype=tf.float64)
# 64비트 실수 텐서를 32비트로 변환하여 실수 텐서와 더하기 -> 가능
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: id=15, shape=(), dtype=float32, numpy=42.0>

# 12.2.4 변수

* tf.Tensor
    - 내용 변환이 불가능한 객체
        - 신경망 가중치 불가 (역전파로 값 업데이트)
        - 모멘텀 옵티마이저 파라미터 불가 (시간에 따라 값 변경)
* tf.Variable
    - tf.tensor와 비슷하게 동작 -> 동일 연산 수행, 넘파이와 호환 가능
    - assign() 사용해서 변수값 변경 가능
        - assign_add(), assing_sub(): 주어진 값 만큼 변수 증가/감소
        - 원소/슬라이스에 assign()가능 -> 직접 수정은 안됨
        - scatter_update(), scatter_nd_update()로 개별 원소/슬라이스 수정 가능

### Variables

In [32]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [29]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [30]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [51]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [31]:
# 원소 직접 수정 안됨
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment


In [33]:
# 여러 원소 수정
v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   2.,   3.],
       [  4.,   5., 200.]], dtype=float32)>

# 12.3 사용자 정의 모델과 훈련 알고리즘

# 12.3.1 사용자 정의 손실 함수

## Custom loss function

### 사용자 정의 손실 함수 정의

In [35]:
# 후버 손실함수
# 공식 케라스 API에서 제공하지 않음 -> tf.keras는 지원(keras.losses.Huber 클래스 사용)
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

### 사용자 정의 손실 함수 적용하기

In [64]:
# 커스텀 손실 함수를 이용해 모델 컴파일
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

In [65]:
# 배치마다 커스텀 손실함수를 호출해 손실 계산 후, 경사 하강법 적용
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 73us/sample - loss: 0.6247 - mae: 0.9969 - val_loss: 0.2884 - val_mae: 0.5913
Epoch 2/2
11610/11610 [==============================] - 0s 32us/sample - loss: 0.2193 - mae: 0.5176 - val_loss: 0.2361 - val_mae: 0.5232


# 12.3.2 사용자 정의 요소를 가진 모델 정의하고 로드하기

## Saving/Loading Models with Custom Objects

### 커스텀 손실 함수 모델 저장하기 

In [38]:
# 케라스가 함수 이름을 저장하므로, 잘 저장됨
model.save("my_model_with_a_custom_loss.h5")

NameError: name 'model' is not defined

### 커스텀 손실 함수 모델 로드하기 

In [40]:
# 모델을 로드할 때, 함수 이름과 실제 함수를 맵핑한 딕셔너리를 전달해야 함
# 커스텀 객체 포함 모델 로드 시, 그 이름과 객체를 매핑 해야함
# custom_objects={"huber_fn": huber_fn} 전달
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [68]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 55us/sample - loss: 0.2056 - mae: 0.4982 - val_loss: 0.2170 - val_mae: 0.5037
Epoch 2/2
11610/11610 [==============================] - 0s 35us/sample - loss: 0.2006 - mae: 0.4911 - val_loss: 0.2097 - val_mae: 0.4908


### 매개변수를 받을 수 있는 커스텀 손실 함수 정의

In [69]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [70]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])

In [71]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 69us/sample - loss: 0.2229 - mae: 0.4893 - val_loss: 0.2525 - val_mae: 0.4973
Epoch 2/2
11610/11610 [==============================] - 0s 32us/sample - loss: 0.2189 - mae: 0.4856 - val_loss: 0.2338 - val_mae: 0.4765


### 매개변수를 받을 수 있는 커스텀 손실 함수 모델 저장하기

In [42]:
# 모델 저장 시, 매개변수 값은 저장 안됨
model.save("my_model_with_a_custom_loss_threshold_2.h5")

### 매개변수를 받을 수 있는 커스텀 손실 함수 모델 로드하기

In [73]:
# 모델 로드시, 매개변수 값을 지정해야 함
# 저장한 케라스 모델에 사용한 함수 이름을 사용해야 함
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [74]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 58us/sample - loss: 0.2148 - mae: 0.4796 - val_loss: 0.2111 - val_mae: 0.4713
Epoch 2/2
11610/11610 [==============================] - 0s 34us/sample - loss: 0.2123 - mae: 0.4775 - val_loss: 0.1970 - val_mae: 0.4534


In [75]:
# keras.lossse.Loss 클래스 상속하고, get_config() 메서드로 구현할 수 있음
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    # 레이블과 예측을 받고 모든 샘플의 손실을 계산하여 반환
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    # 하이퍼파라미터 이름과 같이 매핑된 딕셔너리를 매핑
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [76]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

### 매개변수 받는 손실 클래스의 인스턴스 적용하기

In [77]:
model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

In [78]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 66us/sample - loss: 0.6934 - mae: 0.8818 - val_loss: 0.3288 - val_mae: 0.5451
Epoch 2/2
11610/11610 [==============================] - 0s 32us/sample - loss: 0.2402 - mae: 0.5077 - val_loss: 0.2315 - val_mae: 0.4861


### 매개변수 받는 손실 클래스의 인스턴스를 적용한 모델 저장하기

In [48]:
# 모델 저장 시, 매개변수 값도 같이 저장
model.save("my_model_with_a_custom_loss_class.h5")

### 매개변수 받는 손실 클래스의 인스턴스를 적용한 모델 로드하기

In [80]:
# 모델 로드 시, 클래스 이름과 클래스 자체를 매핑해 주어야 함
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5", # TODO: check PR #25956
                                custom_objects={"HuberLoss": HuberLoss})

In [81]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 0s 33us/sample - loss: 0.2263 - mae: 0.4946 - val_loss: 0.2285 - val_mae: 0.4851
Epoch 2/2
11610/11610 [==============================] - 0s 33us/sample - loss: 0.2217 - mae: 0.4913 - val_loss: 0.1994 - val_mae: 0.4635


In [83]:
model.loss.threshold

2.0

# 12.3.3 활성화 함수, 초기화, 규제, 제한을 커스터마이징하기

* 손실, 규제, 제한, 초기화, 지표, 활성화 함수, 층, 모델 커스터 마이징 하기
* 입출력 함수 정의 후 적용
* 함수가 모델과 함께 저장할 하이퍼라라미터를 가지면 클래스를 상속
    - keras.regularizers.Regularizer
    - keras.constraints.Constraint
    - keras.initializer.Initialzer

## Other Custom Functions

In [84]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [51]:
# 커스텀 활성화 함수
# keras.activations.softplus()와 동일
# tf.nn.softplus()와 동일
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)

# 커스텀 글로럿 초기화 함수
# keras.initializers.glorot_normal()와 동일
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# 커스텀 l1규제 함수
# keras.regularizers.l1(0.01)과 동일
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# 커스텀 제한(양수 가중치만 남기기) 함수
# keras.constraints.noneg() 동일
# tf.relu() 동일
def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [52]:
# dense층에 활성화 함수 적용하여, 다음 층에 전달
# 층의 가중치는 초기화 함수에서 반환된 값으로 초기화
# 훈련 스텝마다 가중치가 규제 함수에 전달되어 규제 손실을 계산하고, 전체 손실에 추가 되어 훈련위한 최종 손실 만ㄷ름
# 훈련 스텝마다 제한 함수가 호출되어 층의 가충치를 제한한 가중치 값으로 변경
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [87]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [88]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1, activation=my_softplus,
                       kernel_regularizer=my_l1_regularizer,
                       kernel_constraint=my_positive_weights,
                       kernel_initializer=my_glorot_initializer),
])

In [89]:
model.compile(loss="mse", optimizer="nadam", metrics=["mae"])

In [90]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 82us/sample - loss: 1.4486 - mae: 0.8727 - val_loss: 2.4208 - val_mae: 0.5681
Epoch 2/2
11610/11610 [==============================] - 0s 40us/sample - loss: 0.5848 - mae: 0.5260 - val_loss: 1.6040 - val_mae: 0.5122


In [91]:
model.save("my_model_with_many_custom_parts.h5")

In [92]:
model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "my_l1_regularizer": my_l1_regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

# 12.3.4 사용자 정의 지표

* 손실(예: 크로스 엔트로피)
    - 모델을 훈련하기 위해 경사 하강법에서 사용하므로, 미분 가능해야하고, 기울기가 0이 아니어야 함
* 지표(예: 정확도)
    - 모델을 평가할 때 사용

## Custom Metrics

In [102]:
# 후버 손실 함수는 지표로 사용 가능
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

### Streaming metrics
- 배치마다 점진적으로 업데이트 됨 
- 상태가 있는 지표(stateful metric)

In [4]:
# 정밀도의 경우, raw값을 저장하여, 배치마다 누적된 raw값으로 계산해야함
# 정밀도: 진짜 양성/ 예측 양성
# 첫번째 배치: 5개 양성 예측, 4개 맞음 -> 정밀도 80%
# 두번째 배치: 3개 양성 예측, 모두 틀림 -> 정밀도 0%
# 평균값: 40%
# 실제 정밀도: 총 8개 양성 예측, 총 4개 맞음 -> 정밀도 50%
precision = keras.metrics.Precision()

In [5]:
# 첫번째 배치 처리 (레이블과 예측값 전달)
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: id=65, shape=(), dtype=float32, numpy=0.8>

In [55]:
# 두번째 배치 처리 (레이블과 예측값 전달)
# 두번째 배치의 정밀도가 아닌, 전체 정밀도 
# 첫번째 배치 정밀도 + 두번째 배치 정밀도 평균이 아님
# 두번째 배치까지의 누적된 raw값을 기반으로 계산
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: id=1838, shape=(), dtype=float32, numpy=0.5>

In [109]:
# result() 현재 지표값 얻기
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [110]:
# 진짜 양성과 거짓 양성 기록 변수 확인
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [111]:
# 지표값 초기화 하기
precision.reset_states()

# 12.3.5 사용자 정의 층

## Custom Layers

### 가중치 없는 층 만들기

In [135]:
# 파이썬 함수 만든 후, keras.laysers.Lambda 층으로 감싸기
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [136]:
exponential_layer([-1., 0., 1.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.36787945, 1.        , 2.7182817 ], dtype=float32)>

In [138]:
# 활성화 함수로 사용하기
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer
])
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=5,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Train on 11610 samples, validate on 3870 samples
Epoch 1/5
11610/11610 [==============================] - 1s 60us/sample - loss: nan - val_loss: nan
Epoch 2/5
11610/11610 [==============================] - 0s 30us/sample - loss: nan - val_loss: nan
Epoch 3/5
11610/11610 [==============================] - 0s 31us/sample - loss: nan - val_loss: nan
Epoch 4/5
11610/11610 [==============================] - 0s 33us/sample - loss: nan - val_loss: nan
Epoch 5/5
5160/5160 [==============================] - 0s 15us/sample - loss: nan


nan

### 가중치(상태가) 있는 층 만들기

In [139]:
# keras.layers.Layer 상속
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    # 층의 변수 만들기
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    # 층에 필요한 연산 수행하기
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

In [141]:
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=input_shape),
    MyDense(1)
])

In [142]:
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Train on 11610 samples, validate on 3870 samples
Epoch 1/2
11610/11610 [==============================] - 1s 60us/sample - loss: 2.1425 - val_loss: 1.2779
Epoch 2/2
5160/5160 [==============================] - 0s 15us/sample - loss: 0.5359


0.5359284550644631

In [143]:
model.save("my_model_with_a_custom_layer.h5")

In [144]:
model = keras.models.load_model("my_model_with_a_custom_layer.h5",
                                custom_objects={"MyDense": MyDense})

# 12.3.6 사용자 정의 모델

* 커스텀 모델 클래스 만들기
    - 서브클래싱 API 사용해 모델 정의
        - keras.Model 클래스 상속
        - 생성자에서 층과 변수 만들기
        - 모델이 해야할 작업을 call()에 구현

## Custom Models

In [151]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

### 커스텀 모델 만들기

In [152]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [154]:
model = ResidualRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
score = model.evaluate(X_test_scaled, y_test)
y_pred = model.predict(X_new_scaled)

Train on 11610 samples
Epoch 1/5
11610/11610 [==============================] - 1s 104us/sample - loss: 11.4379
Epoch 2/5
11610/11610 [==============================] - 0s 41us/sample - loss: 1.7623
Epoch 3/5
11610/11610 [==============================] - 0s 42us/sample - loss: 1.4744
Epoch 4/5
11610/11610 [==============================] - 0s 41us/sample - loss: 0.6444
Epoch 5/5
5160/5160 [==============================] - 0s 32us/sample - loss: 0.6047


In [155]:
model.save("my_custom_model.ckpt")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_custom_model.ckpt/assets


In [156]:
model = keras.models.load_model("my_custom_model.ckpt")

We could have defined the model using the sequential API instead:

In [159]:
block1 = ResidualBlock(2, 30)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    block1, block1, block1, block1,
    ResidualBlock(2, 30),
    keras.layers.Dense(1)
])

# 12.4 텐서플로 함수와 그래프

## TensorFlow Functions

tf.funtion()
- 함수에서 수행하는 계산을 분석하고 동일한 작업을 수행하는 계산 그래프를 생성
- 방법
    - tf.funtion(함수)
    - tf.funtion 데코레이터 사용 -> 더 널리 사용
- 텐서플로는 원본 함수보다 빠르게 실행 
    - 텐서플로는 계산 그래프를 사용하지 않는 노드를 제거하고 표현을 단순화(예. 1+2 -> 3으로 대체)하는 등의 방식으로 최적화 -> 최적화된 그래프가 준비되면 적절한 순서에 가능한 병렬로 그래프 내 연산을 효율적 실행
    - 복잡한 연산일 수록 속도 향상
- 케라스에서는 사용자 정의 손실/지표/층/함수를 모델에 사용할 때 자동으로 텐서플로 함수로 변환
- 텐서를 매개변수로 텐서플로 함수를 호출해야함 (하이퍼파라미터 같이 고유값이 있는 경우만 파이썬 값으로 호출)

In [15]:
# 함수 정의
def cube(x):
    return x ** 3
cube

<function __main__.cube(x)>

In [3]:
# 상수로 함수 호출
cube(2)

8

In [6]:
#텐서로 함수 호출
cube(tf.constant(2.0))

<tf.Tensor: id=2, shape=(), dtype=float32, numpy=8.0>

In [7]:
# tf.funtion(): 함수를 텐서플로 함수로 변경
tf_cube = tf.function(cube)
tf_cube

In [8]:
tf_cube(2)

<tf.Tensor: id=9, shape=(), dtype=int32, numpy=8>

In [9]:
tf_cube(tf.constant(2.0))

<tf.Tensor: id=17, shape=(), dtype=float32, numpy=8.0>

In [14]:
# tf.function() 데코레이터 사용
@tf.function()
def tf_cube(x):
    return x ** 3
tf_cube

In [17]:
# 원본 함수가 필요할 때 python_function() 속성으로 참조 가능
tf_cube.python_function(2)

8